In [9]:
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
from MLGenerateData import MLGenerateData
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import numpy as np 

In [10]:
data = pd.read_csv("./6_24months_US_car_data.csv")
target = data['profit']
predictors = data.drop(['profit'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))


print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.890
Method:                 Least Squares   F-statistic:                     824.5
Date:                Mon, 12 Jun 2023   Prob (F-statistic):               0.00
Time:                        12:37:21   Log-Likelihood:                -33985.
No. Observations:                3164   AIC:                         6.803e+04
Df Residuals:                    3132   BIC:                         6.823e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.402e+04   1471.769      9.525      0.0

/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [11]:
data = pd.read_csv("./6_24months_car_data.csv")
target = data['profit']
predictors = data.drop(['profit'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))


print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     11.95
Date:                Mon, 12 Jun 2023   Prob (F-statistic):           1.09e-51
Time:                        12:37:21   Log-Likelihood:                -25035.
No. Observations:                1343   AIC:                         5.013e+04
Df Residuals:                    1311   BIC:                         5.030e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.853e+06    9.9e+06      0.692      0.4

In [12]:
data = pd.read_csv("./144_US_car_data.csv")
target = data['profit']
predictors = data.drop(['profit'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))


print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     398.0
Date:                Mon, 12 Jun 2023   Prob (F-statistic):               0.00
Time:                        12:37:22   Log-Likelihood:                -33403.
No. Observations:                3164   AIC:                         6.695e+04
Df Residuals:                    3090   BIC:                         6.740e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.358e+04   1253.062     10.834      0.0

In [13]:
data = pd.read_csv("./144_car_data.csv")
target = data['profit']
predictors = data.drop(['profit'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))


print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                     49.17
Date:                Mon, 12 Jun 2023   Prob (F-statistic):          3.37e-316
Time:                        12:37:22   Log-Likelihood:                -22973.
No. Observations:                1343   AIC:                         4.609e+04
Df Residuals:                    1269   BIC:                         4.648e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        8.76e+06   2.21e+06      3.955      0.0